In [1]:
import statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df1 = pd.read_csv("E:\MDA 2023\Full noise data\Jan\csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv",delimiter=';')
df1 = df1[['result_timestamp','laeq']]
df2 = pd.read_csv("E:\MDA 2023\Full noise data\Feb\csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv",delimiter=';')
df2 = df2[['result_timestamp','laeq']]
df3 = pd.read_csv("E:\MDA 2023\Full noise data\March\csv_results_44_255443_mp-06-parkstraat-2-la-filosovia.csv",delimiter=';')
df3 = df3[['result_timestamp','laeq']]
df4 = pd.read_csv("E:\MDA 2023\Full noise data\April\csv_results_45_255443_mp-06-parkstraat-2-la-filosovia.csv",delimiter=';')
df4 = df4[['result_timestamp','laeq']]
df5 = pd.read_csv("E:\MDA 2023\Full noise data\May\csv_results_46_255443_mp-06-parkstraat-2-la-filosovia.csv",delimiter=';')
df5 = df5[['result_timestamp','laeq']]
df6 = pd.read_csv("E:\MDA 2023\Full noise data\June\csv_results_47_255443_mp-06-parkstraat-2-la-filosovia.csv",delimiter=';')
df6 = df6[['result_timestamp','laeq']]
df7 = pd.read_csv("E:\MDA 2023\Full noise data\Jul\csv_results_48_255443_mp-06-parkstraat-2-la-filosovia.csv",delimiter=';')
df7 = df7[['result_timestamp','laeq']]
df8 = pd.read_csv("E:\MDA 2023\Full noise data\Aug\csv_results_49_255443_mp-06-parkstraat-2-la-filosovia.csv",delimiter=';')
df8 = df8[['result_timestamp','laeq']]
df9 = pd.read_csv("E:\MDA 2023\Full noise data\Sep\csv_results_50_255443_mp-06-parkstraat-2-la-filosovia.csv",delimiter=';')
df9 = df9[['result_timestamp','laeq']]
df10 = pd.read_csv("E:\MDA 2023\Full noise data\Oct\csv_results_51_255443_mp-06-parkstraat-2-la-filosovia.csv",delimiter=';')
df10 = df10[['result_timestamp','laeq']]
df11 = pd.read_csv("E:\MDA 2023\Full noise data\\Nov\csv_results_52_255443_mp-06-parkstraat-2-la-filosovia.csv",delimiter=';')
df11 = df11[['result_timestamp','laeq']]
df12 = pd.read_csv("E:\MDA 2023\Full noise data\Dec\csv_results_53_255443_mp-06-parkstraat-2-la-filosovia.csv",delimiter=';')
df12 = df12[['result_timestamp','laeq']]
dfn = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12], axis=0)

In [3]:
# Convert noise datetime column to a Pandas datetime object
dfn['result_timestamp'] = pd.to_datetime(dfn['result_timestamp'], format="%d/%m/%Y %H:%M:%S.%f")
df_backup = dfn

In [4]:
#dfn = df_backup

In [5]:
# set timestamp column as the index
dfn.set_index('result_timestamp', inplace=True)

In [6]:
# resample to one minute frequency and aggregate using the mean
dfn = dfn.resample('10T').mean()

In [7]:
# Drop the datetime column from the index and reset the index to default
dfn = dfn.reset_index()
dfn

,result_timestamp,laeq
0,2022-01-01 00:00:00,54.790667
1,2022-01-01 00:10:00,48.605833
2,2022-01-01 00:20:00,54.277833
3,2022-01-01 00:30:00,50.025333
4,2022-01-01 00:40:00,48.907333
...,...,...
52555,2022-12-31 23:10:00,52.203333
52556,2022-12-31 23:20:00,54.622833
52557,2022-12-31 23:30:00,56.135667
52558,2022-12-31 23:40:00,59.380333


In [8]:
# Extract minute-in-the-hour feature (makes model slightly less accurate)
#dfn['minute'] = dfn['result_timestamp'].dt.minute.astype('int64')

# Extract hour-in-the-day feature
dfn['hour'] = dfn['result_timestamp'].dt.hour.astype('int64')

# Extract day-in-the-week feature
dfn['day'] = dfn['result_timestamp'].dt.dayofweek.astype('int64')

# Extract day-in-the-month feature
dfn['day_of_month'] = dfn['result_timestamp'].dt.day.astype('int64')

# Extract month-in-the-year feature
dfn['month'] = dfn['result_timestamp'].dt.month.astype('int64')

In [9]:
def weekend_column(df):
    is_weekend=lambda x: 1 if x==5 or x==6 else 0
    df["weekend"]=df["day"].apply(is_weekend)
    return(df)
dfn=weekend_column(dfn)

In [10]:
#create numeric columns to merge on
dfn['result_timestamp'] = dfn['result_timestamp'].astype('int64')
dfn

,result_timestamp,laeq,hour,day,day_of_month,month,weekend
0,1640995200000000000,54.790667,0,5,1,1,1
1,1640995800000000000,48.605833,0,5,1,1,1
2,1640996400000000000,54.277833,0,5,1,1,1
3,1640997000000000000,50.025333,0,5,1,1,1
4,1640997600000000000,48.907333,0,5,1,1,1
...,...,...,...,...,...,...,...
52555,1672528200000000000,52.203333,23,5,31,12,1
52556,1672528800000000000,54.622833,23,5,31,12,1
52557,1672529400000000000,56.135667,23,5,31,12,1
52558,1672530000000000000,59.380333,23,5,31,12,1


In [11]:
#get weather data
dfw1 = pd.read_csv("E:\MDA 2023\dataverse_files\LC_2022Q1.csv")
dfw1 = dfw1[dfw1['ID'] == 'LC-105']
dfw2 = pd.read_csv("E:\MDA 2023\dataverse_files\LC_2022Q2.csv")
dfw2 = dfw2[dfw2['ID'] == 'LC-105']
dfw3 = pd.read_csv("E:\MDA 2023\dataverse_files\LC_2022Q3.csv")
dfw3 = dfw3[dfw3['ID'] == 'LC-105']
dfw4 = pd.read_csv("E:\MDA 2023\dataverse_files\LC_2022Q4.csv")
dfw4 = dfw4[dfw4['ID'] == 'LC-105']
dfw = pd.concat([dfw1,dfw2,dfw3,dfw4],axis=0)
dfw = dfw[['DATEUTC', 'LC_HUMIDITY', 'LC_DWPTEMP', 'LC_n', 'LC_RAD', 'LC_RAININ', 'LC_DAILYRAIN','LC_WINDDIR',
          'LC_WINDSPEED', 'LC_RAD60', 'LC_TEMP_QCL0','LC_TEMP_QCL1','LC_TEMP_QCL2','LC_TEMP_QCL3']]

dfw['result_timestamp'] = dfw['DATEUTC']
dfw = dfw.drop(['DATEUTC'], axis=1)
dfw['result_timestamp'] = pd.to_datetime(dfw['result_timestamp'],format="%Y-%m-%d %H:%M:%S")
dfw['result_timestamp'] = dfw['result_timestamp'].astype('int64')
dfw = dfw[dfw.isnull()['LC_HUMIDITY']==False]
dfw = dfw[dfw['result_timestamp'].duplicated()==False]
dfw.duplicated().sum()

0

In [26]:
#merge noise and weather data
df = pd.merge(dfn,dfw,on='result_timestamp', validate="1:1")

In [27]:
df['result_timestamp'] = pd.to_datetime(df['result_timestamp'])
df

,result_timestamp,laeq,hour,day,day_of_month,month,weekend,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,LC_RAD60,LC_TEMP_QCL0,LC_TEMP_QCL1,LC_TEMP_QCL2,LC_TEMP_QCL3
0,2022-01-01 00:10:00,48.605833,0,5,1,1,1,91.0,11.88,38.0,0.0,0.0,0.0,-48.0,0.04,0.0,13.28,13.28,13.223,13.23639
1,2022-01-01 00:20:00,54.277833,0,5,1,1,1,90.0,11.72,37.0,0.0,0.0,0.0,45.0,0.02,0.0,13.28,13.28,13.223,13.16699
2,2022-01-01 00:30:00,50.025333,0,5,1,1,1,90.0,11.72,38.0,0.0,0.0,0.0,-52.0,0.04,0.0,13.28,13.28,13.223,13.19886
3,2022-01-01 00:40:00,48.907333,0,5,1,1,1,90.0,11.72,37.0,0.0,0.0,0.0,34.0,0.02,0.0,13.28,13.28,13.223,13.16699
4,2022-01-01 00:50:00,47.905000,0,5,1,1,1,90.0,11.72,38.0,0.0,0.0,0.0,0.0,0.00,0.0,13.28,13.28,13.223,13.09212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52548,2022-12-31 23:10:00,52.203333,23,5,31,12,1,52.0,6.79,38.0,0.0,0.0,0.0,-9.0,0.40,0.0,16.61,16.61,16.553,16.52797
52549,2022-12-31 23:20:00,54.622833,23,5,31,12,1,52.0,6.74,38.0,0.0,0.0,0.0,5.0,0.29,0.0,16.61,16.61,16.553,16.57995
52550,2022-12-31 23:30:00,56.135667,23,5,31,12,1,52.0,6.62,37.0,0.0,0.0,0.0,-17.0,0.15,0.0,16.51,16.51,16.453,16.44846
52551,2022-12-31 23:40:00,59.380333,23,5,31,12,1,52.0,6.60,38.0,0.0,0.0,0.0,-31.0,0.29,0.0,16.49,16.49,16.433,16.46641


In [28]:
# check for missings
df.isnull().sum()

result_timestamp     0
laeq                84
hour                 0
day                  0
day_of_month         0
month                0
weekend              0
LC_HUMIDITY          0
LC_DWPTEMP           0
LC_n                 0
LC_RAD               0
LC_RAININ            0
LC_DAILYRAIN         0
LC_WINDDIR           0
LC_WINDSPEED         0
LC_RAD60             0
LC_TEMP_QCL0         0
LC_TEMP_QCL1        54
LC_TEMP_QCL2        54
LC_TEMP_QCL3        54
dtype: int64

In [29]:
# only keep rows without missings
df=df[df.isnull()['LC_TEMP_QCL2']==False]
df=df[df.isnull()['laeq']==False]

In [30]:
#doublecheck whether missings are gone
df.isnull().sum()

result_timestamp    0
laeq                0
hour                0
day                 0
day_of_month        0
month               0
weekend             0
LC_HUMIDITY         0
LC_DWPTEMP          0
LC_n                0
LC_RAD              0
LC_RAININ           0
LC_DAILYRAIN        0
LC_WINDDIR          0
LC_WINDSPEED        0
LC_RAD60            0
LC_TEMP_QCL0        0
LC_TEMP_QCL1        0
LC_TEMP_QCL2        0
LC_TEMP_QCL3        0
dtype: int64

In [31]:
# Drop the datetime column
df_nd = df.drop('result_timestamp', axis=1)

In [32]:
df

,result_timestamp,laeq,hour,day,day_of_month,month,weekend,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,LC_RAD60,LC_TEMP_QCL0,LC_TEMP_QCL1,LC_TEMP_QCL2,LC_TEMP_QCL3
0,2022-01-01 00:10:00,48.605833,0,5,1,1,1,91.0,11.88,38.0,0.0,0.0,0.0,-48.0,0.04,0.0,13.28,13.28,13.223,13.23639
1,2022-01-01 00:20:00,54.277833,0,5,1,1,1,90.0,11.72,37.0,0.0,0.0,0.0,45.0,0.02,0.0,13.28,13.28,13.223,13.16699
2,2022-01-01 00:30:00,50.025333,0,5,1,1,1,90.0,11.72,38.0,0.0,0.0,0.0,-52.0,0.04,0.0,13.28,13.28,13.223,13.19886
3,2022-01-01 00:40:00,48.907333,0,5,1,1,1,90.0,11.72,37.0,0.0,0.0,0.0,34.0,0.02,0.0,13.28,13.28,13.223,13.16699
4,2022-01-01 00:50:00,47.905000,0,5,1,1,1,90.0,11.72,38.0,0.0,0.0,0.0,0.0,0.00,0.0,13.28,13.28,13.223,13.09212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52548,2022-12-31 23:10:00,52.203333,23,5,31,12,1,52.0,6.79,38.0,0.0,0.0,0.0,-9.0,0.40,0.0,16.61,16.61,16.553,16.52797
52549,2022-12-31 23:20:00,54.622833,23,5,31,12,1,52.0,6.74,38.0,0.0,0.0,0.0,5.0,0.29,0.0,16.61,16.61,16.553,16.57995
52550,2022-12-31 23:30:00,56.135667,23,5,31,12,1,52.0,6.62,37.0,0.0,0.0,0.0,-17.0,0.15,0.0,16.51,16.51,16.453,16.44846
52551,2022-12-31 23:40:00,59.380333,23,5,31,12,1,52.0,6.60,38.0,0.0,0.0,0.0,-31.0,0.29,0.0,16.49,16.49,16.433,16.46641


In [42]:
df_arima = df.set_index(['result_timestamp'])
df_arima = pd.DataFrame(df_arima['laeq'])
df_arima.index.to_period("10T")

PeriodIndex(['2022-01-01 00:10', '2022-01-01 00:20', '2022-01-01 00:30',
             '2022-01-01 00:40', '2022-01-01 00:50', '2022-01-01 01:00',
             '2022-01-01 01:10', '2022-01-01 01:20', '2022-01-01 01:30',
             '2022-01-01 01:40',
             ...
             '2022-12-31 22:20', '2022-12-31 22:30', '2022-12-31 22:40',
             '2022-12-31 22:50', '2022-12-31 23:00', '2022-12-31 23:10',
             '2022-12-31 23:20', '2022-12-31 23:30', '2022-12-31 23:40',
             '2022-12-31 23:50'],
            dtype='period[10T]', name='result_timestamp', length=52416)

In [43]:
df_arima

,laeq
result_timestamp,
2022-01-01 00:10:00,48.605833
2022-01-01 00:20:00,54.277833
2022-01-01 00:30:00,50.025333
2022-01-01 00:40:00,48.907333
2022-01-01 00:50:00,47.905000
...,...
2022-12-31 23:10:00,52.203333
2022-12-31 23:20:00,54.622833
2022-12-31 23:30:00,56.135667


In [36]:
from statsmodels.tsa.stattools import adfuller
dftest = adfuller(df_arima.laeq, autolag = 'AIC')
print("1. ADF : ",dftest[0])
print("2. P-Value : ", dftest[1])
print("3. Num Of Lags : ", dftest[2])
print("4. Num Of Observations Used For ADF Regression and Critical Values Calculation :", dftest[3])
print("5. Critical Values :")
for key, val in dftest[4].items():
    print("\t",key, ": ", val)

1. ADF :  -32.04925202256659
2. P-Value :  0.0
3. Num Of Lags :  50
4. Num Of Observations Used For ADF Regression and Critical Values Calculation : 52365
5. Critical Values :
	 1% :  -3.430474885335369
	 5% :  -2.8615951968083673
	 10% :  -2.566799379426004


In [45]:
from statsmodels.tsa.seasonal import seasonal_decompose
decompose_df_arima = seasonal_decompose(df_arima, model="additive")
decompose_df_arima.plot()

TypeError: float() argument must be a string or a number, not 'Period'

In [ ]:
class statsmodels.tsa.statespace.sarimax.SARIMAX(
    y, exog=X, order=(1, 0, 0), seasonal_order=(0, 0, 0, 0),
    trend=None, measurement_error=False, time_varying_regression=False,
    mle_regression=True, simple_differencing=False, enforce_stationarity=True,
    enforce_invertibility=True, hamilton_representation=False, concentrate_scale=False,
    trend_offset=1, use_exact_diffuse=False, dates=None, freq=None, missing='none',
    validate_specification=True, **kwargs)